# Оглавление

0) Получение данных - Процессинг - Итерация по списку портфелей

1) Поиск похожих портфелей по каждому искомому портфелю

2) Оценка эффективности похожих портфелей для каждого искомого портфеля

3) Доп критерии фильтрации (Частота инвестирования, Риск-профиль итд)

4) Рекомендация активов, которых нет в искомом портфеле, но есть в эффективных похожих, учитываю платежеспособность пользователя

# 0) Получение данных - Процессинг - Итерация по списку портфелей

In [ ]:
import pyodbc
import pandas as pd
pd.set_option('display.max_rows', 500)

connection_DB = pyodbc.connect(driver = '{SQL Server}', server = 'bd-srv-apl' , database = 'StoreHouse',
                         user='xxx', password='xxx')

In [ ]:
connection_DB_CRM = pyodbc.connect(Trusted_Connection='yes', driver = '{SQL Server}', server = 'BD-SRV-DWH' , database = 'OpenDWH')

query1 = "select * from [Data].[ClientPortfolio]"
df_all_portf = pd.read_sql(query1, connection_DB)

In [ ]:
# фильтрация по последней календарной дате
last_date = df_all_portf['CourierTime'].max()
df_all_portf = df_all_portf[df_all_portf['CourierTime'] == last_date]
df_all_portf.fillna(0,inplace=True)

In [ ]:
# преобразование столбца с абсолютными значениями в относительные по портфелю
sum_table = df_all_portf.groupby(['Portfolio_Id'])['Amount'].sum().reset_index().rename(columns={'Amount': 'percents'})
df_all_portf = df_all_portf.merge(sum_table, on='Portfolio_Id', how = 'left')

df_all_portf['Amount_Absolute'] = df_all_portf['Amount']
df_all_portf['Amount']  = (df_all_portf['Amount'] / df_all_portf['percents'])
df_all_portf['Amount'] = df_all_portf['Amount'].apply(lambda x: round(x, 2))
float_list = [float(0) if e == float("inf") or e == float("-inf") else float(e) for e in list(df_all_portf['Amount']) ]
df_all_portf['Amount'] = float_list

# 1) Поиск похожих портфелей по каждому искомому портфелю


In [ ]:
# поиск похожих

import numpy as np
import scipy.sparse as sparse
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import NearestNeighbors
from tqdm import tqdm_notebook

data = df_all_portf.groupby(['Portfolio_Id','SecurityName'],as_index=False)['Amount'].sum()
data['Portfolio_Id'] = data['Portfolio_Id'].astype(int)

users = list(np.sort(data.Portfolio_Id.unique()))
le_users = LabelEncoder()
users_l = le_users.fit_transform(users)

secs = list(np.sort(data.SecurityName.unique()))
le_secs = LabelEncoder()
secs_l = le_secs.fit_transform(secs)

counts = list(data.Amount)

rows = list(le_secs.transform(data.SecurityName.astype(str)))
cols = list(le_users.transform(data.Portfolio_Id.astype(int)))

data_sparse = sparse.csr_matrix((counts,(cols, rows)), shape=(max(cols)+1, max(rows)+1))

from sklearn.neighbors import NearestNeighbors

neigh = NearestNeighbors(n_neighbors = 10, n_jobs=-1, metric='euclidean')
neigh.fit(data_sparse)


In [ ]:
# итерация по каждому портфелю, чтобы найти список похожих

neighboors_list = []
for e in tqdm_notebook(range(0, int(max(cols)))):
    neighboors_list.append(neigh.kneighbors(data_sparse[e].toarray()[0].reshape(1, -1))[1])

# 2) Оценка эффективности похожих портфелей для каждого искомого портфеля

In [ ]:
import pandas as pd
import warnings
import numpy as np
warnings.filterwarnings('ignore')
pd.set_option('max_colwidth', 100000000000)
    
#Подгрузка финансовых данных из СУБД и процессинг

coub_old = pd.read_excel('Coub1_2016_2018_сВыводом.xlsx')
coub_old.fillna(0, inplace=True)
coub_new = pd.read_excel('Coub1_2019_сВыводом.xlsx')
coub_new.fillna(0,inplace=True)
df_COUB_GUID_Dirty = pd.concat([coub_old, coub_new], axis=0)
del coub_old
del coub_new

df_COUB_GUID = df_COUB_GUID_Dirty[~df_COUB_GUID_Dirty['GUID'].isnull()]
df_COUB_GUID = df_COUB_GUID[df_COUB_GUID['GUID']!="<...>"]
df_COUB_GUID.fillna(0,inplace=True)
coub_data = df_COUB_GUID[['GUID', 'Год', 'Месяц','Активы руб на конец П', 'ДС+ЦБ Ввод руб','ДС+ЦБ Вывод руб', 'Фин рез П без НДС']] 

dict_russian_month = {'Январь': 1, 'Февраль': 2, 'Март': 3, 'Апрель': 4, 'Май': 5, 'Июнь': 6, 'Июль': 7, 'Август': 8, 'Сентябрь': 9, 'Октябрь': 10, 'Ноябрь': 11, 'Декабрь': 12}
coub_data.replace({"Месяц": dict_russian_month},inplace=True)
coub_data['date'] = coub_data['Месяц'].astype(str) +'/'+ coub_data['Год'].astype(str)
coub_data['date'] = pd.to_datetime(coub_data['date'])
coub_data.sort_values(['GUID','date'], ascending=[False,True], inplace=True)
#Создание ключевой переменной для сравнивания прироста рын стоимости активов в портфеле по месяцем
coub_data['Активы очищенные на конец П'] = coub_data['Активы руб на конец П'] - coub_data['ДС+ЦБ Ввод руб'] + coub_data['ДС+ЦБ Вывод руб'] - coub_data['Фин рез П без НДС']
coub_data['Активы руб на конец П'] = coub_data['Активы руб на конец П'].astype(int)
coub_data['Активы очищенные на конец П'] = coub_data['Активы очищенные на конец П'].astype(int)



In [ ]:
guid_list_1 = coub_data.groupby('GUID')['Активы очищенные на конец П'].apply(list).to_frame()
guid_list_2 = coub_data.groupby('GUID')['Активы руб на конец П'].apply(list).to_frame()
guid_list = guid_list_1.merge(guid_list_2,on='GUID',how='left').reset_index()

In [ ]:
from statistics import mean 
def Average(lst): 
    return mean(lst) 

guid_list['avg'] = guid_list['Активы руб на конец П'].apply(lambda x: Average(x))
guid_list['period'] = guid_list['Активы руб на конец П'].apply(lambda x: len(x))

In [ ]:
actives_on_end = list(guid_list_corr['Активы руб на конец П'])
actives_on_end_clear = list(guid_list_corr['Активы очищенные на конец П'])

delta_col_abs = []
pos_count = []
rating_list = []
for ind,e in enumerate(actives_on_end):
    tmp_abs = [(b - a) for a, b in zip(e, actives_on_end_clear[ind][1:])] #Нахожу абсолютный прирост
    delta_col_abs.append(tmp_abs[1:])
    pos_count.append(sum([1 for x in tmp_abs[1:] if x > 0])) #Считаю кол-во положительных приростов. Отсекаю [1:], так как первый месяц шумный
    rating_list.append(sum([1 for x in tmp_abs[1:] if x > 0])/median) # Чтобы ранжировать по формуле ((число положительных приростов/ длину листа) * (длину листа/медианная длина листа по столбцу))
    
guid_list_corr['delta'] = delta_col_abs
guid_list_corr['pos_count'] = pos_count
guid_list_corr['grade'] = rating_list

In [ ]:
guid_list_corr.sort_values('grade', ascending=False).head(1)

# 4) Рекомендация активов, которых нет в искомом портфеле, но есть в эффективных похожих, учитываю платежеспособность пользователя

In [ ]:
import pandas as pd
import pyodbc

connection_DB_CRM = pyodbc.connect(Trusted_Connection='yes', driver = '{SQL Server}',server = 'BD-SRV-DWH' , database = 'OpenDWH')

#Подгрузка последнего месяца из CRM-ки 
sql_crm="SELECT * FROM [datamartMarketing].[vwCrmProspect]"
SQL_CRM_all = pd.read_sql(sql_crm,connection_DB_CRM)

In [ ]:
cbcodes

In [ ]:
guids = []
for e in cbcodes[1:]:
    guids.append(list(SQL_CRM_all[SQL_CRM_all['client_base_code'] == e]['GUID'].unique()))

In [ ]:
guids

In [ ]:
df_totals = []
for i,e in enumerate(guids):
    if e != []:
        for j in e:
            df = guid_list_corr[guid_list_corr['GUID'] == j]
            df['clientbasecode'] = cbcodes[1:][i]
            df['Portfolio_Id'] = df_all_portf[df_all_portf['ClientCode'] == cbcodes[1:][i]]['Portfolio_Id'].unique()[0]
            df_totals.append(df)

In [ ]:
df_totals_concat = pd.concat(df_totals,axis=0).sort_values('pos_count',ascending=False)
df_totals_concat = df_totals_concat[df_totals_concat['pos_count'] > 0]

In [ ]:
secs_user = data[data['Portfolio_Id'] == 44444444444]['SecurityName'].unique()
recom_user = data[(data['Portfolio_Id'] == 4444444444448)]
recom_user[~recom_user['SecurityName'].isin(secs_user)]

Отсев предлагаемых акций с учетом платежеспособности искомого пользователя

In [ ]:
from urllib.parse import urlencode
from urllib.request import urlopen
from datetime import datetime
#пользовательские переменные
ticker="SBER" #задаём тикер
period=7 # задаём период. Выбор из: 'tick': 1, 'min': 2, '5min': 3, '10min': 4, '15min': 5, '30min': 6, 'hour': 7, 'daily': 8, 'week': 9, 'month': 10
start = "19.02.2020" #с какой даты начинать тянуть котировки
end = "20.02.2020" #финальная дата, по которую тянуть котировки
########
periods={'tick': 1, 'min': 2, '5min': 3, '10min': 4, '15min': 5, '30min': 6, 'hour': 7, 'daily': 8, 'week': 9, 'month': 10}
print ("ticker="+ticker+"; period="+str(period)+"; start="+start+"; end="+end)
#каждой акции Финам присвоил цифровой код:
tickers={'ABRD':82460,'AESL':181867,'AFKS':19715,'AFLT':29,'AGRO':399716,'AKRN':17564,'ALBK':82616,'ALNU':81882,'ALRS':81820,'AMEZ':20702,'APTK':13855,'AQUA':35238,'ARMD':19676,'ARSA':19915,'ASSB':16452,'AVAN':82843,'AVAZ':39,'AVAZP':40,'BANE':81757,'BANEP':81758,'BGDE':175840,'BISV':35242,'BISVP':35243,'BLNG':21078,'BRZL':81901,'BSPB':20066,'CBOM':420694,'CHEP':20999,'CHGZ':81933,'CHKZ':21000,'CHMF':16136,'CHMK':21001,'CHZN':19960,'CLSB':16712,'CLSBP':16713,'CNTL':21002,'CNTLP':81575,'DASB':16825,'DGBZ':17919,'DIOD':35363,'DIXY':18564,'DVEC':19724,'DZRD':74744,'DZRDP':74745,'ELTZ':81934,'ENRU':16440,'EPLN':451471,'ERCO':81935,'FEES':20509,'FESH':20708,'FORTP':82164,'GAZA':81997,'GAZAP':81998,'GAZC':81398,'GAZP':16842,'GAZS':81399,'GAZT':82115,'GCHE':20125,'GMKN':795,'GRAZ':16610,'GRNT':449114,'GTLC':152876,'GTPR':175842,'GTSS':436120,'HALS':17698,'HIMC':81939,'HIMCP':81940,'HYDR':20266,'IDJT':388276,'IDVP':409486,'IGST':81885,'IGST03':81886,'IGSTP':81887,'IRAO':20516,'IRGZ':9,'IRKT':15547,'ISKJ':17137,'JNOS':15722,'JNOSP':15723,'KAZT':81941,'KAZTP':81942,'KBSB':19916,'KBTK':35285,'KCHE':20030,'KCHEP':20498,'KGKC':83261,'KGKCP':152350,'KLSB':16329,'KMAZ':15544,'KMEZ':22525,'KMTZ':81903,'KOGK':20710,'KRKN':81891,'KRKNP':81892,'KRKO':81905,'KRKOP':81906,'KROT':510,'KROTP':511,'KRSB':20912,'KRSBP':20913,'KRSG':15518,'KSGR':75094,'KTSB':16284,'KTSBP':16285,'KUBE':522,'KUNF':81943,'KUZB':83165,'KZMS':17359,'KZOS':81856,'KZOSP':81857,'LIFE':74584,'LKOH':8,'LNTA':385792,'LNZL':21004,'LNZLP':22094,'LPSB':16276,'LSNG':31,'LSNGP':542,'LSRG':19736,'LVHK':152517,'MAGE':74562,'MAGEP':74563,'MAGN':16782,'MERF':20947,'MFGS':30,'MFGSP':51,'MFON':152516,'MGNT':17086,'MGNZ':20892,'MGTS':12984,'MGTSP':12983,'MGVM':81829,'MISB':16330,'MISBP':16331,'MNFD':80390,'MOBB':82890,'MOEX':152798,'MORI':81944,'MOTZ':21116,'MRKC':20235,'MRKK':20412,'MRKP':20107,'MRKS':20346,'MRKU':20402,'MRKV':20286,'MRKY':20681,'MRKZ':20309,'MRSB':16359,'MSNG':6,'MSRS':16917,'MSST':152676,'MSTT':74549,'MTLR':21018,'MTLRP':80745,'MTSS':15523,'MUGS':81945,'MUGSP':81946,'MVID':19737,'NAUK':81992,'NFAZ':81287,'NKHP':450432,'NKNC':20100,'NKNCP':20101,'NKSH':81947,'NLMK':17046,'NMTP':19629,'NNSB':16615,'NNSBP':16616,'NPOF':81858,'NSVZ':81929,'NVTK':17370,'ODVA':20737,'OFCB':80728,'OGKB':18684,'OMSH':22891,'OMZZP':15844,'OPIN':20711,'OSMP':21006,'OTCP':407627,'PAZA':81896,'PHOR':81114,'PHST':19717,'PIKK':18654,'PLSM':81241,'PLZL':17123,'PMSB':16908,'PMSBP':16909,'POLY':175924,'PRFN':83121,'PRIM':17850,'PRIN':22806,'PRMB':80818,'PRTK':35247,'PSBR':152320,'QIWI':181610,'RASP':17713,'RBCM':74779,'RDRB':181755,'RGSS':181934,'RKKE':20321,'RLMN':152677,'RLMNP':388313,'RNAV':66644,'RODNP':66693,'ROLO':181316,'ROSB':16866,'ROSN':17273,'ROST':20637,'RSTI':20971,'RSTIP':20972,'RTGZ':152397,'RTKM':7,'RTKMP':15,'RTSB':16783,'RTSBP':16784,'RUAL':414279,'RUALR':74718,'RUGR':66893,'RUSI':81786,'RUSP':20712,'RZSB':16455,'SAGO':445,'SAGOP':70,'SARE':11,'SAREP':24,'SBER':3,'SBERP':23,'SELG':81360,'SELGP':82610,'SELL':21166,'SIBG':436091,'SIBN':2,'SKYC':83122,'SNGS':4,'SNGSP':13,'STSB':20087,'STSBP':20088,'SVAV':16080,'SYNG':19651,'SZPR':22401,'TAER':80593,'TANL':81914,'TANLP':81915,'TASB':16265,'TASBP':16266,'TATN':825,'TATNP':826,'TGKA':18382,'TGKB':17597,'TGKBP':18189,'TGKD':18310,'TGKDP':18391,'TGKN':18176,'TGKO':81899,'TNSE':420644,'TORS':16797,'TORSP':16798,'TRCN':74561,'TRMK':18441,'TRNFP':1012,'TTLK':18371,'TUCH':74746,'TUZA':20716,'UCSS':175781,'UKUZ':20717,'UNAC':22843,'UNKL':82493,'UPRO':18584,'URFD':75124,'URKA':19623,'URKZ':82611,'USBN':81953,'UTAR':15522,'UTII':81040,'UTSY':419504,'UWGN':414560,'VDSB':16352,'VGSB':16456,'VGSBP':16457,'VJGZ':81954,'VJGZP':81955,'VLHZ':17257,'VRAO':20958,'VRAOP':20959,'VRSB':16546,'VRSBP':16547,'VSMO':15965,'VSYD':83251,'VSYDP':83252,'VTBR':19043,'VTGK':19632,'VTRS':82886,'VZRZ':17068,'VZRZP':17067,'WTCM':19095,'WTCMP':19096,'YAKG':81917,'YKEN':81766,'YKENP':81769,'YNDX':388383,'YRSB':16342,'YRSBP':16343,'ZHIV':181674,'ZILL':81918,'ZMZN':556,'ZMZNP':603,'ZVEZ':82001}
FINAM_URL = "http://export.finam.ru/"# сервер, на который стучимся
market = 0 #можно не задавать. Это рынок, на котором торгуется бумага. Для акций работает с любой цифрой. Другие рынки не проверял.
#Делаем преобразования дат:
start_date = datetime.strptime(start, "%d.%m.%Y").date()
start_date_rev=datetime.strptime(start, '%d.%m.%Y').strftime('%Y%m%d')
end_date = datetime.strptime(end, "%d.%m.%Y").date()
end_date_rev=datetime.strptime(end, '%d.%m.%Y').strftime('%Y%m%d')
#Все параметры упаковываем в единую структуру. Здесь есть дополнительные параметры, кроме тех, которые заданы в шапке. См. комментарии внизу:
params = urlencode([
					('market', market), #на каком рынке торгуется бумага
					('em', tickers[ticker]), #вытягиваем цифровой символ, который соответствует бумаге.
					('code', ticker), #тикер нашей акции
					('apply',0), #не нашёл что это значит. 
					('df', start_date.day), #Начальная дата, номер дня (1-31)
					('mf', start_date.month - 1), #Начальная дата, номер месяца (0-11)
					('yf', start_date.year), #Начальная дата, год
					('from', start_date), #Начальная дата полностью
					('dt', end_date.day), #Конечная дата, номер дня	
					('mt', end_date.month - 1), #Конечная дата, номер месяца
					('yt', end_date.year), #Конечная дата, год
					('to', end_date), #Конечная дата
					('p', period), #Таймфрейм
					('f', ticker+"_" + start_date_rev + "_" + end_date_rev), #Имя сформированного файла
					('e', ".csv"), #Расширение сформированного файла
					('cn', ticker), #ещё раз тикер акции	
					('dtf', 1), #В каком формате брать даты. Выбор из 5 возможных. См. страницу https://www.finam.ru/profile/moex-akcii/sberbank/export/
					('tmf', 1), #В каком формате брать время. Выбор из 4 возможных.
					('MSOR', 0), #Время свечи (0 - open; 1 - close)	
					('mstime', "on"), #Московское время	
					('mstimever', 1), #Коррекция часового пояса	
					('sep', 1), #Разделитель полей	(1 - запятая, 2 - точка, 3 - точка с запятой, 4 - табуляция, 5 - пробел)
					('sep2', 1), #Разделитель разрядов
					('datf', 1), #Формат записи в файл. Выбор из 6 возможных.
					('at', 1)]) #Нужны ли заголовки столбцов
url = FINAM_URL + ticker+"_" + start_date_rev + "_" + end_date_rev + ".csv?" + params #урл составлен!
print("Стучимся на Финам по ссылке: "+url)
txt=urlopen(url).readlines() #здесь лежит огромный массив данных, прилетевший с Финама.
local_file = open('quotes.txt', "w") #задаём файл, в который запишем котировки.
for line in txt: #записываем свечи строку за строкой. 
	local_file.write(line.strip().decode( "utf-8" )+'\n')	
local_file.close()
print("Готово. Проверьте файл quotes.txt в папке где лежит скрипт")